# Un WebSocket Client écrit en Rust en WebAssembly

---

## Introduction

Dans cette partie, nous allons réaliser le site du client afin que l'on puisse utiliser notre echo serveur. Enfin, plus exactement, nous réaliserons surtout la partie web assembly en Rust pour gérer l'API WEB WebSocket via la crate web_sys de wasm_bindgen.

> L'objectif de ce chapitre est de présenter les différents raisonnements qui seraient nécessaires à l'utilisation de lAPI WebSocket dans le cadre du WebAssembly en Rust.

## Documentation

> - La première ressource dont nous aurons besoin est la description de l'API WEB WebSocket de Mozilla. Je vous en donne donc le lien [https://developer.mozilla.org/en-US/docs/Web/API/WebSockets_API/Writing_WebSocket_client_applications](https://developer.mozilla.org/en-US/docs/Web/API/WebSockets_API/Writing_WebSocket_client_applications) :

![API](./pictures/websocket-api.png)

> - La seconde ressource est celle du binding de l'api WebSocket de la documentation Rust (obtenue via `cargo doc --open`) de la structure WebSocket de la crate web_sys :

![WS](./pictures/doc-websys-ws.png)

*Cela constitue les deux documentations essentielles a la réalisation du client,* **veuillez ne pas regarder l'exemple des websockets du guide de wasm-bindgen pour vous en inspire, ce serait tricher.** 

## Création d'un nouveau projet

> Afin de ne pas perdre du temps a créer un nouveau projet, je vous suggère de copier le **boilerplate** wasm que j'ai écrit, il est ici `~/Desktop/programs/wasm-boilerplate` :

```
cp -r ~/Desktop/programs/wasm-boilerplate ~/Desktop/client-ws
cd ~/Desktop/client-ws
```

*Afin de pouvoir compiler le code Rust en web assembly puis de lancer la page web, utilisez ces deux scripts depuis la racine du projet :*

`./install.sh && ./run.sh`

## Les preliminaires

- Afin d'avoir un vrai nouveau projet, il convient de changer le nom du projet dans le fichier `Cargo.toml` a la racine.
-  vous suggère très fortement de tout de suite installer les 3 crates qui nous permettront d'avoir des logs de meilleure qualité ainsi que les panic dans la console WEB.
```
[ To add dependencies ...]
wasm-logger = "0.2.0"
log = "0.4.6"
console_error_panic_hook = "0.1.7"   
```
- Enfin, dans la fonction `start` de votre programme rust, initialisez les de suite :
```
#[wasm_bindgen(start)]
fn ma_fonction_start() -> Result<(), JsValue> {
    wasm_logger::init(wasm_logger::Config::default());
    std::panic::set_hook(Box::new(console_error_panic_hook::hook));
    [...]
```
- Dernière chose, d'après la documentation de l'API WebSocket via web_sys, la feature `WebSocket` est à activer, ajoutez le à la liste des features :
```
[dependencies.web-sys]
version = "0.3.63"
features = [ 
    ... others features ...
    WebSocket
]
```

> *Voilà, nous sommes maintenant prêts à passer à l'écriture du code.*

## Principe général

Ce que l'on vise ici, c'est la possibilité de pouvoir envoyer un message au serveur puis de lire sa réponse. Comme il s'agit d'un echo server, le message reçu sera identique à ce que l'on a envoyé.

D'après la documentation de l'API Websocket, il est question de faire un `new` au départ du programme afin d'ouvrir une websocket vers le serveur. Ensuite, viendront des implémentations de callback telles que `on_open`, `on_error` & `on_message`. Nous aurons aussi besoin d'utiliser la méthode `send` pour pouvoir écrire un message au serveur. Ce send se fera une fois la socket ouverte, donc dans la callback de open.

## Implémentations

### La création de la socket

La première étape sera de créer une nouvelle WebSocket via la méthode new, d'après la doc de l'api, elle prend en paramètre une chaine de caractère contenant l'IP ou l'URL suivi du numéro du port séparés par le symbole `:`.

- Regardons maintenant ce que cela donne du côté de l'api rust web_sys :

>```
>impl WebSocket
>pub fn new(url: &str) -> Result<WebSocket, JsValue>
>
>The new WebSocket(..) constructor, creating a new instance of WebSocket.
>
>MDN Documentation
>
>This API requires the following crate features to be activated: WebSocket
>
>```
*Bon, je pense que vous êtes en mesure de pouvoir coder ça sans problème….*

### L'implémentation du listener `open`

Une fois la websocket prête avec `new`, un événement `open` devrait normalement survenir.

```
The open event is fired when a connection with a WebSocket is opened.
```

La section correspondante dans web_sys est :
>```
>impl WebSocket
>
>pub fn set_onopen(&self, value: Option<&Function>)
>
>Setter for the onopen field of this object.
>
>MDN Documentation
>
>This API requires the following crate features to be activated: WebSocket
>
>```

Cette méthode s'appelle donc en lui passant une Option de callback en paramètre, nous avons vu dans le précédent chapitre comment créer une closure de type `Closure` et comment en faire une `Fonction` Javascript.

**méthode new :**

>```
>impl<T> Closure<T>
>where
>    T: ?Sized + WasmClosure,
>
>pub fn new<F>(t: F) -> Closure<T>
>where
>    F: IntoWasmClosure<T> + 'static,
>
>Creates a new instance of Closure from the provided Rust function.
>
>Note that the closure provided here, F, has a few requirements associated with it:
>
>    It must implement Fn or FnMut (for FnOnce functions see Closure::once and Closure::once_into_js).
>
>    It must be 'static, aka no stack references (use the move keyword).
>
>    It can have at most 7 arguments.
>
>    Its arguments and return values are all types that can be shared with JS (i.e. have #[wasm_bindgen] >annotations or are simple numbers, etc.)
>```

**type conversion :**

`{Type Function}` = `{Type Closure}.as_ref().unchecked_ref())`

**NB : Vous devez aussi trouver un moyen d'éviter que votre closure soit déchargée de la mémoire au moment où vous sortez du bloc de code dans laquelle elle est déclarée !**

### L'implémentation du listener `error`

La création de ce listener est très semblable à open, cependant un Event d'erreur peut être générée et lue dans la callback, et il peut être intéressant de penser à le faire. Le type ws_sys d'erreur event est `ErrorEvent`.

### L'implémentation du listener `message`

Pour ce dernier listener, ce sera un Event de type `MessageEvent` qui sera retourné, il existe une méthode de MessageEvent qui permet de récupérer les données, et un cast en chaine de caractère de type `js_sys::JsString` peut être utile dans la mesure où l'on s'attend normalement a une chaine de caractère.

![EVT](./pictures/msg_event.png)

### `Écrire un message` au serveur

La crate ws_sys définit la méthode `send_with_str` qui peut correspondre à ce dont on a besoin :

>```
>impl WebSocket
>
>pub fn send_with_str(&self, data: &str) -> Result<(), JsValue>
>
>The send() method.
>
>MDN Documentation
>
>This API requires the following crate features to be activated: WebSocket
>
>```

## Conclusion

![CO](./pictures/console.png)

**Utilisé conjointement avec le serveur rust codé dans la partie précédente, vous devriez être normalement en mesure de recevoir la chaine de caractère que vous avec envoyé au serveur.**